In [5]:
import pandas as pd
import tensorflow as tf
import numpy as np
import time

/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
bz = pd.read_csv('yelp_dataset/restaurant.csv',index_col='Unnamed: 0')
rv = pd.read_csv('yelp_dataset/reviews_filtered.csv',index_col='Unnamed: 0')
us = pd.read_csv('yelp_dataset/user_filtered.csv',index_col='Unnamed: 0')

# bz --> rv --> us

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19,51,78,88,89,90,91) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [13]:
# bz1 = bz.sample(frac=0.1)
# rv1 = bz1[['business_id']].merge(rv, how='left',on='business_id')
# us1 = rv1[['user_id']].merge(rv1, how='left',on='user_id')

In [16]:
df = rv.groupby(['business_id','user_id']).agg({'stars':'mean'}).reset_index()

In [19]:
df.head()

,business_id,user_id,stars
0,--9e1ONYQuAa-CB_Rrw7Tw,---1lKK3aKOuomHnwAkAow,4.0
1,--9e1ONYQuAa-CB_Rrw7Tw,-11nMat3F0DgsxT30hn8zA,5.0
2,--9e1ONYQuAa-CB_Rrw7Tw,-3fMaL_ck0wzEsTZyz-mqA,5.0
3,--9e1ONYQuAa-CB_Rrw7Tw,-47EprTYhe9IbAvplGIcpA,5.0
4,--9e1ONYQuAa-CB_Rrw7Tw,-7cPERHtAupd1Xq53nmhXg,5.0


In [25]:
df_train['user_id'].nunique()

373483

In [26]:
df_train['business_id'].nunique()

8848

In [24]:
msk = np.random.rand(len(df)) < 0.7
df_train = df[msk]
user_ind = [x-1 for x in df_train['user_id'].index]
item_ind = [x-1 for x in df_train['business_id'].index]
stars = df_train['stars'].index

In [27]:
# variables
feature_len = 10
U = tf.Variable(initial_value=tf.truncated_normal([373483,feature_len]), name='users')
P = tf.Variable(initial_value=tf.truncated_normal([feature_len,8848]), name='items')
result = tf.matmul(U, P)
result_flatten = tf.reshape(result, [-1])

Instructions for updating:
Colocations handled automatically by placer.


In [29]:
R = tf.gather(result_flatten, user_ind * tf.shape(result)[1] + 
              item_ind, name='extracting_user_stars')

In [32]:
# cost function
diff_op = tf.subtract(R, stars, name='training_diff')
diff_op_squared = tf.abs(diff_op, name="squared_difference")
base_cost = tf.reduce_sum(diff_op_squared, name="sum_squared_error")

TypeError: Expected float32 passed to parameter 'y' of op 'Sub', got Int64Index([      2,       3,       4,       6,       7,       8,       9,
                 11,      12,      13,
            ...
            1344729, 1344732, 1344735, 1344736, 1344737, 1344738, 1344739,
            1344740, 1344741, 1344743],
           dtype='int64', length=941587) of type 'Int64Index' instead.

In [ ]:
# regularization
lda = tf.constant(.001, name='lambda')
norm_sums = tf.add(tf.reduce_sum(tf.abs(U, name='user_abs'), name='user_norm'), 
   tf.reduce_sum(tf.abs(T, name='item_abs'), name='item_norm'))
regularizer = tf.mul(norm_sums, lda, 'regularizer')

cost = tf.add(base_cost, regularizer)

In [ ]:
# optimizer
lr = tf.constant(.001, name='learning_rate')
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(lr, global_step, 10000, 0.96, staircase=True)
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_step = optimizer.minimize(cost, global_step=global_step)

In [ ]:
# execution
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)
for i in xrange(500):
    sess.run(training_step)


In [ ]:
# testing
u, p, r = df[['user', 'item', 'rate']].values[0]
rhat = tf.gather(tf.gather(result, u-1), p-1)
print "rating for user " + str(u) + " for item " + str(p) + 
      " is " + str(r) + " and our prediction is: " + str(sess.run(rhat))

In [ ]:
# calculate accuracy
df_test = df[~msk]
user_indecies_test = [x-1 for x in df_test.user.values]
item_indecies_test = [x-1 for x in df_test.item.values]
rates_test = df_test.rate.values

# accuracy
R_test = tf.gather(result_flatten, user_indecies_test * tf.shape(result)[1] + item_indecies_test, name='extracting_user_rate_test')
diff_op_test = tf.sub(R_test, rates_test, name='test_diff')
diff_op_squared_test = tf.abs(diff_op, name="squared_difference_test")

cost_test = tf.div(tf.reduce_sum(tf.square(diff_op_squared_test, name="squared_difference_test"), name="sum_squared_error_test"), df_test.shape[0] * 2, name="average_error")
print sess.run(cost_test)